In [162]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from langdetect import detect

In [163]:
import string
import glob
import nltk
import math
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kollu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kollu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kollu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [164]:
#case folding
def case_folding(lines):
  lines=lines.casefold()
  return lines
#removing punctuations
def rm_punctuation(words):
  words=words.translate(str.maketrans('','',string.punctuation))
  return words
#tkenization
def tokenization(words):
  token=nltk.word_tokenize(words)
  return token
#removing single letter tokens
def rm_single(token):
  for i in token:
    if len(i)==1:
      token.remove(i)
  return token
#removing stop words
def rm_stop_words(token):
  stop_words = set(stopwords.words('english'))
  for word in token:
    if word in stop_words:
        token.remove(word)
  return token
#stemming of tokens
def stemming(token):
  porter = PorterStemmer()
  for i in range(0,len(token)):
    token[i]=porter.stem(token[i])
  return token
#lemmatizing of tokens
def lemmatizing(token):
  for i in range(0,len(token)):
    token[i]=WordNetLemmatizer().lemmatize(token[i])
  return token

In [165]:
def pre_processing(text): 
    text=case_folding(text)
    text=rm_punctuation(text)
    tokens=tokenization(text)
    tokens=rm_single(tokens)
    tokens=rm_stop_words(tokens)
    tokens=stemming(tokens)
    tokens=lemmatizing(tokens)
    return tokens

In [166]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

# load the model and the feature extraction object
with open('spam_model.pkl', 'rb') as file:
    model = pickle.load(file)

with open('vectorizer.pkl', 'rb') as file:
    feature_extraction = pickle.load(file)

# define new input text
new_input = ["I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times"]

# transform the new input text to feature vectors using the saved feature extraction object
new_input_features = feature_extraction.transform(new_input)

# make predictions on the new input using the trained model
predictions = model.predict(new_input_features)

if predictions[0] == 1:
    print('Ham mail')
else:
    print('Spam mail')


Ham mail


In [167]:
v={35: 'U.S. NEWS', 5: 'COMEDY', 22: 'PARENTING', 40: 'WORLD NEWS', 7: 'CULTURE & ARTS', 32: 'TECH', 28: 'SPORTS', 10: 'ENTERTAINMENT', 24: 'POLITICS', 37: 'WEIRD NEWS', 11: 'ENVIRONMENT', 9: 'EDUCATION', 6: 'CRIME', 27: 'SCIENCE', 38: 'WELLNESS', 3: 'BUSINESS', 30: 'STYLE & BEAUTY', 13: 'FOOD & DRINK', 20: 'MEDIA', 25: 'QUEER VOICES', 17: 'HOME & LIVING', 39: 'WOMEN', 2: 'BLACK VOICES', 34: 'TRAVEL', 21: 'MONEY', 26: 'RELIGION', 19: 'LATINO VOICES', 18: 'IMPACT', 36: 'WEDDINGS', 4: 'COLLEGE', 23: 'PARENTS', 1: 'ARTS & CULTURE', 29: 'STYLE', 15: 'GREEN', 31: 'TASTE', 16: 'HEALTHY LIVING', 33: 'THE WORLDPOST', 14: 'GOOD NEWS', 41: 'WORLDPOST', 12: 'FIFTY', 0: 'ARTS', 8: 'DIVORCE'}

In [168]:
query = "ms dhoni is the famous cricketer"
limit = 5000
tweets = []
import pickle

with open('classify_model.pkl', 'rb') as f:
    category_model = pickle.load(f)

with open('spam_model.pkl', 'rb') as f:
    spam_model = pickle.load(f)

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        if detect(tweet.content)=='en':
            spam_prediction = spam_model.predict(feature_extraction.transform([tweet.content]))
            if spam_prediction[0] == 1:
                spam_pre='Ham'
            else:
                spam_pre='Spam'
            category=v[category_model.predict([tweet.content])[0]]
            tweets.append([tweet.date, tweet.username, tweet.content,tweet.url, tweet.user.followersCount, tweet.likeCount, tweet.retweetCount,category,spam_pre])

df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet','Url', 'Followers', 'Likes', 'Retweets','Category','Spam/Ham'])
print(df)

c:\Users\kollu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\kollu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\kollu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MultinomialNB from version 1.2.2 when u

                        Date             User  \
0  2023-04-07 11:52:39+00:00       vfxexpress   
1  2023-01-14 06:23:01+00:00  RohitSi04769888   
2  2022-11-26 05:50:14+00:00          ady2939   
3  2022-11-22 09:59:41+00:00       _hindutvaa   
4  2022-11-17 15:09:03+00:00       wwemeister   
5  2022-11-16 05:30:34+00:00       wwemeister   
6  2022-10-12 11:15:45+00:00        e_monthly   
7  2022-10-03 12:45:42+00:00      Trademarkia   
8  2022-07-07 14:34:57+00:00        CcrKaphle   
9  2022-06-05 17:08:22+00:00        _vikram18   
10 2022-05-10 23:08:17+00:00        DeStonkAI   
11 2022-03-27 13:08:20+00:00  _vivek_mishra07   
12 2021-11-16 09:54:10+00:00    ElitecricketP   
13 2021-11-14 16:58:58+00:00   joydiphazarika   
14 2021-10-01 15:00:50+00:00  Rajath_Viratian   
15 2021-07-14 18:05:33+00:00      TUnlimitedd   
16 2021-07-12 10:47:20+00:00      Karthik3895   
17 2021-07-11 05:58:32+00:00      Karthik3895   
18 2021-07-11 05:01:16+00:00      Karthik3895   
19 2021-06-18 16:05:

In [169]:
import re
#Creating a function to do the preprocessing: case folding, tokenization, stopword removal, lemmatization and stemming
nltk_stopwords = nltk.corpus.stopwords.words('english')

def preprocess(text):
  
    #Case Folding: convert all characters to lowercase
    text = text.lower()
    #Remove punctuation
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)

    #Tokenization: using own tokenizer
    text = text.strip()
    tokens = re.split('\W+', text)

    #Remove empty tokens and single character tokens
    tokens = [token for token in tokens if token != '' and len(token) > 1]

    #Stopword Removal: remove stopwords like 'a', 'the', 'is', etc.
    tokens = [token for token in tokens if token not in nltk_stopwords]
    #Lemmatization: convert words to their base form (e.g. 'better' to 'good')
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    #Stemming: convert words to their root form (e.g. 'caring' to 'care', 'launched' to 'launch', etc.)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

#Applying the function to the dataframe
df['Content_preprocessed'] = df['Tweet'].apply(preprocess)
df.head()

,Date,User,Tweet,Url,Followers,Likes,Retweets,Category,Spam/Ham,Content_preprocessed
0,2023-04-07 11:52:39+00:00,vfxexpress,Unacademy Commercial VFX Breakdown\nBy Famous ...,https://twitter.com/vfxexpress/status/16443071...,2166,1,0,ENTERTAINMENT,Ham,"[unacademi, commerci, vfx, breakdown, famou, s..."
1,2023-01-14 06:23:01+00:00,RohitSi04769888,hi @jiocare @reliancejio @airtelindia @Airtel_...,https://twitter.com/RohitSi04769888/status/161...,4,1,0,WEIRD NEWS,Ham,"[hi, jiocar, reliancejio, airtelindia, airtelp..."
2,2022-11-26 05:50:14+00:00,ady2939,@SportyVishaI Kuch bhi. IPL would have been fa...,https://twitter.com/ady2939/status/15963807912...,86,3,1,RELIGION,Ham,"[sportyvishai, kuch, bhi, ipl, would, famou, k..."
3,2022-11-22 09:59:41+00:00,_hindutvaa,"@Imro_fanclub greatest batsman, there is no on...",https://twitter.com/_hindutvaa/status/15949940...,40,3,0,STYLE & BEAUTY,Ham,"[imrofanclub, greatest, batsman, one, near, sa..."
4,2022-11-17 15:09:03+00:00,wwemeister,@MyWorldPod @RealJeffJarrett Loved the podcast...,https://twitter.com/wwemeister/status/15932599...,99,1,0,WORLD NEWS,Ham,"[myworldpod, realjeffjarrett, love, podcast, s..."


In [170]:
'''
The given question asks us to apply the vector space model to the corpus with different weighting schemes: 
1. For document:
    a. For term frequency: logaritmic
    b. For document frequency: none
    c. Normalization: cosine

2. For quey:
    a. For term frequency: logaritmic
    b. For document frequency: t ie inverse document frequency
    c. Normalization: cosine
'''

#Defining some basic functions for the vector space model
def calculate_idf(N: int, df: int) -> float:
    '''
    calculate the inverse document frequency (idf) of a term
    :param N: number of documents
    :param df: document frequency of a term (number of documents containing the term)
    '''
    if df == 0:
        return 0
    return math.log(N/df, 10)

def get_document_frequency(term: str, documents: list) -> int:
    '''
    calculate the document frequency of a term in a list of documents
    :param term: the term
    :param documents: the list of documents
    '''
    count = 0
    for document in documents:
        if term in document:
            count += 1
    return count

def log_term_frequency(freq: int) -> float:
    '''
    calculate the log term frequency of a term in a document
    :param freq: the term frequency
    '''
    if freq == 0:
        return 0
    return 1 + math.log(freq, 10)


In [171]:
def preprocess_documents(df):
    document_vectors = []
    for document in df['Content_preprocessed']:
        #Calculating the term frequency 
        term_frequency = {}
        for term in document:
            if term in term_frequency:
                term_frequency[term] += 1
            else:
                term_frequency[term] = 1
        #Calculating the log term frequency 1.a
        for term in term_frequency:
            term_frequency[term] = log_term_frequency(term_frequency[term])
        #Calculating the document frequency 1.b, but not used because document frequency is not part of the weighting scheme
        #Taking cosine normalization 1.c
        sum_of_squares = sum([freq**2 for freq in term_frequency.values()])
        sq_sum_of_squares = math.sqrt(sum_of_squares)
        for term in term_frequency:
            term_frequency[term] /= sq_sum_of_squares
            
        document_vectors.append(term_frequency)

    return document_vectors

dc_vectors = preprocess_documents(df)

In [172]:
def preprocess_query(query: str) -> list:
    '''
    preprocess the query
    :param query: the query
    '''

    #Preprocessing the query
    query = preprocess(query)
    #Calculating the term frequency of the query
    query_vector = {}
    for term in query:
        if term in query_vector:
            query_vector[term] += 1
        else:
            query_vector[term] = 1
    #Calculating the inverse document frequency of the query terms 2.b
    idf = {}
    for term in query_vector:
        idf[term] = calculate_idf(len(df), get_document_frequency(term, df['Content_preprocessed']))
    #Calculating the log term frequency of the query terms 2.a
    for term in query_vector:
        query_vector[term] = log_term_frequency(query_vector[term])
    #Multiplying the term frequency with the inverse document frequency 2.b
    for term in query_vector:
        query_vector[term] = query_vector[term] * idf[term]

    #Normalizing the query vector with cosine normalization 2.c
    sum_of_squares = sum([freq**2 for freq in query_vector.values()])
    sq_sum_of_squares = math.sqrt(sum_of_squares)
    if sq_sum_of_squares == 0:
        return query_vector
    for term in query_vector:
        query_vector[term] /= sq_sum_of_squares

    return query_vector

In [173]:
def calculate_cosine_similarity(query_vector: dict, document_vector: dict) -> float:
    '''
    calculate the cosine similarity between a query and a document
    :param query_vector: the query vector
    :param document_vector: the document vector
    '''
    numerator = 0
    for term in query_vector:
        if term in document_vector:
            numerator += query_vector[term] * document_vector[term]
    return numerator

def get_top_k_documents(query: str, k: int) -> list:
    '''
    get the top k documents for a query
    :param query: the query
    :param k: number of documents to return
    '''
    output=[]
    query_vector = preprocess_query(query)
    scores = []
    for document_vector in dc_vectors:
        scores.append(calculate_cosine_similarity(query_vector, document_vector))
    output=scores    
    print(scores)
    #Printing the top k documents with their titles and scores
    #Getting the indices of the top k documentslo
    #tweet-content,twewt-likes,tweet-retweets,tweet-content.username,tweet url,followers,category
    top_k_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    for i in range(k):
        print('\n' + str(i+1) + ')\nTitle: ', df.iloc[top_k_indices[i]]['Tweet'])
        print('Score: ', scores[top_k_indices[i]], end="\n---------------XXX---------------")
    return output

In [174]:
output=get_top_k_documents(query, len(df))

[0.265939208845558, 0.36344463743064237, 0.31562035933045207, 0.3166527678504192, 0.2527298745170079, 0.2527298745170079, 0.34359082073152836, 0.0, 0.0, 0.3049158717051089, 0.386382176966171, 0.0, 0.0, 0.2919445793625196, 0.12343214428901547, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2662131774573004, 0.36193393121342415, 0.10790403380941468, 0.27121097803319644, 0.0, 0.2869685809273892, 0.36730633818425745, 0.29998390735037644, 0.31151127000499734, 0.33332003958076245, 0.40085802342945376, 0.23652107381280507, 0.0, 0.26349818609446596, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1202010981162486, 0.12068185683572928, 0.1465652673619122, 0.0, 0.34828018774423264, 0.16812328012830527, 0.0, 0.0, 0.27321345746240017, 0.0, 0.0, 0.10312360301913635, 0.0, 0.30625303433234297, 0.0, 0.5265501504812145, 0.44747300972927373, 0.44747300972927373, 0.42004171078623814, 0.46437358365897685]

1)
Title:  MS Dhoni is the most famous cricketer in the world.....
 https://t.co/bt8cU7ouyI https://t.co/aDwpkJeDs8
Score:  0.5265501504

In [175]:
print(output)

[0.265939208845558, 0.36344463743064237, 0.31562035933045207, 0.3166527678504192, 0.2527298745170079, 0.2527298745170079, 0.34359082073152836, 0.0, 0.0, 0.3049158717051089, 0.386382176966171, 0.0, 0.0, 0.2919445793625196, 0.12343214428901547, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2662131774573004, 0.36193393121342415, 0.10790403380941468, 0.27121097803319644, 0.0, 0.2869685809273892, 0.36730633818425745, 0.29998390735037644, 0.31151127000499734, 0.33332003958076245, 0.40085802342945376, 0.23652107381280507, 0.0, 0.26349818609446596, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1202010981162486, 0.12068185683572928, 0.1465652673619122, 0.0, 0.34828018774423264, 0.16812328012830527, 0.0, 0.0, 0.27321345746240017, 0.0, 0.0, 0.10312360301913635, 0.0, 0.30625303433234297, 0.0, 0.5265501504812145, 0.44747300972927373, 0.44747300972927373, 0.42004171078623814, 0.46437358365897685]


In [176]:
df['Scores'] = output
df.head()

,Date,User,Tweet,Url,Followers,Likes,Retweets,Category,Spam/Ham,Content_preprocessed,Scores
0,2023-04-07 11:52:39+00:00,vfxexpress,Unacademy Commercial VFX Breakdown\nBy Famous ...,https://twitter.com/vfxexpress/status/16443071...,2166,1,0,ENTERTAINMENT,Ham,"[unacademi, commerci, vfx, breakdown, famou, s...",0.265939
1,2023-01-14 06:23:01+00:00,RohitSi04769888,hi @jiocare @reliancejio @airtelindia @Airtel_...,https://twitter.com/RohitSi04769888/status/161...,4,1,0,WEIRD NEWS,Ham,"[hi, jiocar, reliancejio, airtelindia, airtelp...",0.363445
2,2022-11-26 05:50:14+00:00,ady2939,@SportyVishaI Kuch bhi. IPL would have been fa...,https://twitter.com/ady2939/status/15963807912...,86,3,1,RELIGION,Ham,"[sportyvishai, kuch, bhi, ipl, would, famou, k...",0.315620
3,2022-11-22 09:59:41+00:00,_hindutvaa,"@Imro_fanclub greatest batsman, there is no on...",https://twitter.com/_hindutvaa/status/15949940...,40,3,0,STYLE & BEAUTY,Ham,"[imrofanclub, greatest, batsman, one, near, sa...",0.316653
4,2022-11-17 15:09:03+00:00,wwemeister,@MyWorldPod @RealJeffJarrett Loved the podcast...,https://twitter.com/wwemeister/status/15932599...,99,1,0,WORLD NEWS,Ham,"[myworldpod, realjeffjarrett, love, podcast, s...",0.252730


In [177]:
import os

filename = 'search_result.csv'

if os.path.isfile(filename):
    with open(filename, 'w', newline='') as f:
        f.write('')
        print("contents cleared.")
else:
    print("error")

contents cleared.


In [178]:
df.to_csv('Search_result.csv', mode='a', index=False, header=False)

In [179]:
# ['Date', 'User', 'Tweet', 'Url', 'Followers', 'Likes', 'Retweets','Category', 'Spam/Ham','Score']